# Extract. Transform. Analysis.

In [1]:
import pandas as pd
import requests
import country_converter as coco
from bs4 import BeautifulSoup
import folium

За основу беру таблицу Numbeo "Current Cost of Living Index", в которой на данный момент 506 городов и, пока, не понятно сколько стран и по сколько городов в этих странах.

In [2]:
url = 'https://www.numbeo.com/cost-of-living/rankings_current.jsp'
df = pd.read_html(url)[1]
df.head()

,Rank,City,Cost of Living Index,Rent Index,Cost of Living Plus Rent Index,Groceries Index,Restaurant Price Index,Local Purchasing Power Index
0,NaN,"Hamilton, Bermuda",161.36,102.29,133.41,167.32,147.19,88.42
1,NaN,"Zurich, Switzerland",121.25,61.89,93.17,122.95,115.77,115.64
2,NaN,"Basel, Switzerland",119.82,45.46,84.64,125.54,114.14,109.13
3,NaN,"Zug, Switzerland",118.68,64.27,92.94,120.79,118.36,136.38
4,NaN,"Bern, Switzerland",113.85,36.83,77.41,115.93,106.74,116.09


Добавляю другие интексы Numbeo относящиеся к городам. В разных таблицах индексы дублируются, но охватывают разное количество городов. Выбираю те, в которых больше городов.

In [3]:
indexes_dict = {
    'https://www.numbeo.com/quality-of-life/rankings_current.jsp': 
        ['City',
         'Quality of Life Index',
         'Property Price to Income Ratio',
         'Traffic Commute Time Index',
         'Climate Index'],
    'https://www.numbeo.com/crime/rankings_current.jsp': ['City', 'Safety Index'],
    'https://www.numbeo.com/health-care/rankings_current.jsp': ['City', 'Health Care Index'],
    'https://www.numbeo.com/pollution/rankings_current.jsp': ['City', 'Pollution Index']
}

In [4]:
for key, value in indexes_dict.items():
    df_temp = pd.read_html(key)[1][value]
    df = df.merge(df_temp, on='City', how='left')

In [5]:
df.drop('Rank', axis=1, inplace=True)
df.columns = list(map(lambda x: x.removesuffix(' Index').lower().replace(' ', '_'), df.columns))
df.rename(columns={'city': 'city_country'}, inplace=True)

In [6]:
df

,city_country,cost_of_living,rent,cost_of_living_plus_rent,groceries,restaurant_price,local_purchasing_power,quality_of_life,property_price_to_income_ratio,traffic_commute_time,climate,safety,health_care,pollution
0,"Hamilton, Bermuda",161.36,102.29,133.41,167.32,147.19,88.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Zurich, Switzerland",121.25,61.89,93.17,122.95,115.77,115.64,194.58,9.95,32.62,81.48,82.68,74.84,17.58
2,"Basel, Switzerland",119.82,45.46,84.64,125.54,114.14,109.13,NaN,NaN,NaN,NaN,79.89,NaN,NaN
3,"Zug, Switzerland",118.68,64.27,92.94,120.79,118.36,136.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Bern, Switzerland",113.85,36.83,77.41,115.93,106.74,116.09,NaN,NaN,NaN,NaN,82.02,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,"Lahore, Pakistan",19.54,3.77,12.08,16.48,14.44,22.19,103.22,14.04,33.94,67.56,64.65,62.52,78.82
506,"Islamabad, Pakistan",19.48,5.13,12.69,16.99,16.12,28.43,139.37,11.49,34.93,76.91,70.70,64.23,41.65
507,"Karachi, Pakistan",19.21,4.16,12.09,17.05,12.74,24.87,81.49,15.46,41.48,71.32,45.97,57.09,89.89
508,"Rawalpindi, Pakistan",18.13,3.28,11.10,16.39,13.63,27.54,NaN,NaN,NaN,NaN,51.90,NaN,NaN


Добавляю в таблицу ссылки на страницы данных городов со строгим указанием валюты USD

In [7]:
response = requests.get(url).text
soup = BeautifulSoup(response, 'lxml')
links = soup.find_all('td', class_='cityOrCountryInIndicesTable')

href_list = []
for i in links:
    href_list.append([i.find('a').text, i.find('a').get('href') + '?displayCurrency=USD'])
    
city_link = pd.DataFrame(href_list, columns=['city_country', 'link'])

df = df.merge(city_link, on='city_country', how='left')

### Разделить Город - Штат - Страна

In [8]:
split_table1 = df['city_country'].str.rsplit(', ', n=1, expand=True)
split_table1.columns=['city_state', 'country']

split_table2 = split_table1['city_state'].str.split(', ', n=1, expand=True)
split_table2.columns=['city', 'state']

df = pd.concat([split_table2, split_table1['country'], df],axis=1)

df['city'] = df.apply(lambda row: row.city.split(' (')[0], axis = 1) #убираю дубли городов в скобках

In [9]:
df.head()

,city,state,country,city_country,cost_of_living,rent,cost_of_living_plus_rent,groceries,restaurant_price,local_purchasing_power,quality_of_life,property_price_to_income_ratio,traffic_commute_time,climate,safety,health_care,pollution,link
0,Hamilton,None,Bermuda,"Hamilton, Bermuda",161.36,102.29,133.41,167.32,147.19,88.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.numbeo.com/cost-of-living/in/Hamil...
1,Zurich,None,Switzerland,"Zurich, Switzerland",121.25,61.89,93.17,122.95,115.77,115.64,194.58,9.95,32.62,81.48,82.68,74.84,17.58,https://www.numbeo.com/cost-of-living/in/Zuric...
2,Basel,None,Switzerland,"Basel, Switzerland",119.82,45.46,84.64,125.54,114.14,109.13,NaN,NaN,NaN,NaN,79.89,NaN,NaN,https://www.numbeo.com/cost-of-living/in/Basel...
3,Zug,None,Switzerland,"Zug, Switzerland",118.68,64.27,92.94,120.79,118.36,136.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.numbeo.com/cost-of-living/in/Zug?d...
4,Bern,None,Switzerland,"Bern, Switzerland",113.85,36.83,77.41,115.93,106.74,116.09,NaN,NaN,NaN,NaN,82.02,NaN,NaN,https://www.numbeo.com/cost-of-living/in/Bern?...


### Добавляю alpha_3 код к странам
Добавляю через lambda к каждой строке. Не самый эффективный вариант, обрабатывается капждая строка, страны дублируются, но просто попробовать (merge уже был).

In [10]:
df['country_code'] = df.apply(lambda row: coco.convert(names=row.country, to='ISO3') , axis = 1)
df.head(3)

,city,state,country,city_country,cost_of_living,rent,cost_of_living_plus_rent,groceries,restaurant_price,local_purchasing_power,quality_of_life,property_price_to_income_ratio,traffic_commute_time,climate,safety,health_care,pollution,link,country_code
0,Hamilton,None,Bermuda,"Hamilton, Bermuda",161.36,102.29,133.41,167.32,147.19,88.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.numbeo.com/cost-of-living/in/Hamil...,BMU
1,Zurich,None,Switzerland,"Zurich, Switzerland",121.25,61.89,93.17,122.95,115.77,115.64,194.58,9.95,32.62,81.48,82.68,74.84,17.58,https://www.numbeo.com/cost-of-living/in/Zuric...,CHE
2,Basel,None,Switzerland,"Basel, Switzerland",119.82,45.46,84.64,125.54,114.14,109.13,NaN,NaN,NaN,NaN,79.89,NaN,NaN,https://www.numbeo.com/cost-of-living/in/Basel...,CHE


### Добавляю код к штатам USA
Есть три значения штатов, не относящихся к штатам США.
Всем этим значениям присвою знанчение None.
А к штатам США добавлю "US- " - приведу к ISO стандарту обозначения штатов США.

In [11]:
df[df['state'].notnull() & (df['country'] != "United States")]

,city,state,country,city_country,cost_of_living,rent,cost_of_living_plus_rent,groceries,restaurant_price,local_purchasing_power,quality_of_life,property_price_to_income_ratio,traffic_commute_time,climate,safety,health_care,pollution,link,country_code
55,Nanaimo,BC,Canada,"Nanaimo, BC, Canada",77.47,37.20,58.42,81.15,66.41,90.51,NaN,NaN,NaN,NaN,50.77,68.65,NaN,https://www.numbeo.com/cost-of-living/in/Nanai...,CAN
73,St. John's,Newfoundland and Labrador,Canada,"St. John's, Newfoundland and Labrador, Canada",75.16,22.95,50.46,74.43,71.00,141.04,NaN,NaN,NaN,NaN,53.06,NaN,NaN,https://www.numbeo.com/cost-of-living/in/St-Jo...,CAN
452,Batumi,Ajara,Georgia,"Batumi, Ajara, Georgia",32.25,12.79,23.04,31.50,27.24,26.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.numbeo.com/cost-of-living/in/Batum...,GEO


In [12]:
indexes = df[df['state'].notnull() & (df['country'] != "United States")]['state'].index
df.loc[indexes, 'state'] = None

df['state'] = df.apply(lambda row: f"US-{row['state']}" if row['state'] != None else None, axis = 1)

# Карта
В индексе Numbeo 505 городов, но не понятно, какие страны, много их или мало. Сколько городов в этих странах и в каких пропорциях. Суммарные цифры мне мало что скажут, гораздо понятнее будет отобразить их на карте.

In [13]:
map_df = df.groupby('country_code')['city'].nunique().reset_index().sort_values('city', ascending=False)

m = folium.Map(location=[37.87820990704326, 6.555063556986549], zoom_start=1.5)
folium.Choropleth(
    geo_data='data/world.geojson',
    name="choropleth",
    data=map_df,
    columns=['country_code', 'city'],
    key_on="feature.properties.ISO_A3", 
    fill_color="YlGn",
    nan_fill_color='pink',
    fill_opacity=0.8,
    bins=7,
    reset=True,
    highlight=True,
    legend_name='Count of cities'
).add_to(m)

![title](data/map.png)

На github интерактивная карта не отображается, поэтому вместо вывода карты, вставил картинку.

Теперь видно какие страны есть в индексе, а каких нет, и сколько городов в каждой стране. Меня всё устраивает, вот только мне не нужно в Индии так много городов. Удалю лишние (оставлю 2).

In [14]:
india = df.loc[(df['country'] == 'India') & 
              (df['city'] != 'Delhi') & 
              (df['city'] !='Mumbai')]

df = df.drop(india.index)

Подготовлю (индексы), чтобы проще было загрузить данные в БД, и сохраню эту таблицу, чтобы потом с нее по каждому городу скрапить данные в БД.

In [15]:
df.reset_index(drop=True, inplace=True)
df.index += 1
df.index.name = 'city_id'

df.to_csv("data/numbeo_main_table.csv")

# Таблица всех штатов США в формате ISO
Достану из википедии и сохраню таблицу кодов штатов США, тк это уже готовая таблица для будущей базы данных.

In [16]:
url_states = 'https://en.wikipedia.org/wiki/ISO_3166-2:US'

df_states = pd.read_html(url_states)[0]

df_states.columns = ['state_code', 'state_name', 'category']
df_states.set_index('state_code', inplace=True)

df_states.to_csv("data/states_codes.csv")

# Таблица всех стран и кодов Alpha 3

In [17]:
url_countries = 'https://en.wikipedia.org/wiki/ISO_3166-1'

df_countries = pd.read_html(url_countries)[1]
df_countries.rename(columns={'English short name (using title case)': 'country', 
                             'Alpha-3 code': 'country_code', 
                             'Independent': 'independent'}, inplace=True)
df_countries = df_countries.iloc[:, [0, 2, 5]]
df_countries.set_index('country_code', inplace=True)

for index, row in df_countries.iterrows():
    if index != 'VGB' and index != 'VIR':
        row['country'] = row['country'].strip('"')
        row['country'] = row['country'].split(' (')[0]
        row['country'] = row['country'].split('[')[0]
        row['country'] = row['country'].split(',')[0]
        
df_countries.to_csv("data/countries_codes.csv")